## Import Library

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

## Import CSV

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/online_retail_II.csv",  header = 0)
df

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom
...,...,...,...,...,...,...,...,...
1067366,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France
1067367,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France
1067368,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France
1067369,581587,22138,BAKING SET 9 PIECE RETROSPOT,3,2011-12-09 12:50:00,4.95,12680.0,France


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1067371 entries, 0 to 1067370
Data columns (total 8 columns):
 #   Column       Non-Null Count    Dtype  
---  ------       --------------    -----  
 0   Invoice      1067371 non-null  object 
 1   StockCode    1067371 non-null  object 
 2   Description  1062989 non-null  object 
 3   Quantity     1067371 non-null  int64  
 4   InvoiceDate  1067371 non-null  object 
 5   Price        1067371 non-null  float64
 6   Customer ID  824364 non-null   float64
 7   Country      1067371 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 65.1+ MB


## Data Cleaning

In [ ]:
df_resik = df.copy()

In [ ]:
# Start Begin
df_resik = df.copy()

# Remove of Description
df_resik = df_resik[~df_resik['Description'].isna()]

# Make all item in description column with no capital
df_resik['Description'] = df_resik['Description'].str.lower()

# Remove all name in the row, that name p, w and post
df_resik = df_resik[~df_resik['StockCode'].str.match('^\d+$', case=False, na=False)]

# Adding order status
df_resik['order_status'] = np.where(df_resik['Invoice'].str[:1]=='C', 'Cancelled', 'Delivered')

# change negatif to positif in quantity
df_resik['Quantity'] = df_resik['Quantity'].abs()

# Remove row that there's negative
df_resik = df_resik[df_resik['Price'] > 0 ]

# Create new column by name amount
df_resik['Amount'] = df_resik['Quantity'] * df_resik['Price']

most_common_product_name = df_resik.groupby(['StockCode', 'Description'], as_index=False) \
  .agg(order_cnt=('Invoice', 'nunique')).sort_values(['StockCode', 'order_cnt'], ascending=[True, False])
most_common_product_name['rank'] = most_common_product_name.groupby('StockCode')['order_cnt'].rank(method = 'first', ascending = False)
most_common_product_name = most_common_product_name.rename(columns={'order_cnt': 'order_cnt', 'rank': 'rank'})
most_common_product_name = most_common_product_name[most_common_product_name['rank'] == 1].drop(columns=['order_cnt', 'rank'])

df_resik = df_resik.merge(
    most_common_product_name.rename(columns={'Description' : '_final_name'}),
    how = 'left',
    on = 'StockCode'
)

df_resik['Description'] = df_resik['_final_name']

# Remove all need help of column
for kolom in ['most_common_product_name','_final_name'] :
  if kolom in df_resik.columns :
    df_resik = df_resik.drop(columns=kolom)

# Change Customer Id to String
df_resik['Customer ID'] = df['Customer ID'].astype(str)
df_resik = df_resik.reset_index(drop= True)

# Lihat Hasil Akhir
print(df_resik.shape)
df_resik.head()

/tmp/ipython-input-5-385454429.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_resik['Description'] = df_resik['Description'].str.lower()
/tmp/ipython-input-5-385454429.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_resik['order_status'] = np.where(df_resik['Invoice'].str[:1]=='C', 'Cancelled', 'Delivered')
/tmp/ipython-input-5-385454429.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead



(133116, 10)


,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,order_status,Amount
0,489434,79323P,pink cherry lights,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,Delivered,81.0
1,489434,79323W,white cherry lights,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,Delivered,81.0
2,489436,48173C,doormat black flock,10,2009-12-01 09:06:00,5.95,13085.0,United Kingdom,Delivered,59.5
3,489436,35004B,set of 3 black flying ducks,12,2009-12-01 09:06:00,4.65,13085.0,United Kingdom,Delivered,55.8
4,489436,84596F,small marshmallows pink bowl,8,2009-12-01 09:06:00,1.25,13085.0,United Kingdom,Delivered,10.0


## EDA (Exsploratory Data Analyst)




In [ ]:
df_resik.shape

(133116, 10)

In [ ]:
df_resik.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 133116 entries, 0 to 133115
Data columns (total 10 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Invoice       133116 non-null  object 
 1   StockCode     133116 non-null  object 
 2   Description   133116 non-null  object 
 3   Quantity      133116 non-null  int64  
 4   InvoiceDate   133116 non-null  object 
 5   Price         133116 non-null  float64
 6   Customer ID   133116 non-null  object 
 7   Country       133116 non-null  object 
 8   order_status  133116 non-null  object 
 9   Amount        133116 non-null  float64
dtypes: float64(2), int64(1), object(7)
memory usage: 10.2+ MB


## Knowing the unique values of categorical columns/variables

In [ ]:
df_resik['order_status'].unique()

array(['Delivered', 'Cancelled'], dtype=object)

In [ ]:
df_resik['order_status'].value_counts()

,count
order_status,
Delivered,129692
Cancelled,3424


## Descriptive count statistic from DataFrame

In [ ]:
df_resik.describe()

,Quantity,Price,Amount
count,133116.000000,133116.000000,133116.000000
mean,9.668710,15.120461,31.216913
std,57.047994,269.692843,278.684892
min,1.000000,0.001000,0.001000
25%,1.000000,1.250000,3.750000
50%,2.000000,2.550000,8.500000
75%,8.000000,4.250000,17.850000
max,6696.000000,38970.000000,38970.000000


## know statistic in every her self

In [ ]:
df_resik['Invoice'].count()

np.int64(133116)

In [ ]:
# Unique Frequency
df_resik['Invoice'].nunique()

28882

In [ ]:
# SUM From quantity
df_resik['Quantity'].sum()

np.int64(1287060)

In [ ]:
# Subset (SUM) From Dataset
df_resik.loc[df_resik['Description'].str.contains('light'),['Quantity']].sum()

,0
Quantity,193544


In [ ]:
# Mean
df_resik['Quantity'].mean()

np.float64(9.668709997295592)

In [ ]:
# Subset (Mean) from dataset
df_resik.loc[df_resik['Description'].str.contains('light'),['Quantity']].mean()

,0
Quantity,12.71058


In [ ]:
# Median
df_resik['Quantity'].median()

2.0

In [ ]:
# Subset Median from dataset
df_resik.loc[df_resik['Description'].str.contains('light'), ['Quantity']].median()

,0
Quantity,6.0


In [ ]:
# Mode
df_resik['Quantity'].mode()

,Quantity
0,1


In [ ]:
# Subset (Mode) From dataset
df_resik.loc[df_resik['Description'].str.contains('light'), ['Quantity']].mode()

,Quantity
0,1


In [ ]:
# Range
df_resik['Quantity'].max() - df_resik['Quantity'].min()

6695

In [ ]:
# Range from subset data
df_resik.loc[df_resik['Description'].str.contains('light'), ['Quantity']] \
.max() - df_resik.loc[df_resik['Description'].str.contains('light'), ['Quantity']].min()

,0
Quantity,1929


In [ ]:
# STD
df_resik['Quantity'].std()

57.04799428057296

In [ ]:
# Deviation Standart range
df_resik.loc[df_resik['Description'].str.contains('light'), ['Quantity']].std()

,0
Quantity,42.004078


In [ ]:
# Variance
df_resik['Quantity'].var()

3254.4736514362853

In [ ]:
df_resik.loc[df_resik['Description'].str.contains('light'), ['Quantity']].var()

,0
Quantity,1764.342541


In [ ]:
# quantile
df_resik['Quantity'].quantile(.25)

np.float64(1.0)

In [ ]:
df_resik.loc[df_resik['Description'].str.contains('light'), ['Quantity']].quantile()

,0.5
Quantity,6.0


In [ ]:
# range interkuartile from data subset
def interquartile_range(x) :
  q1 = x.quantile(.25)
  q2 = x.quantile(.75)
  return q2 - q1

## count statistic descriptif by grouping

In [ ]:
df_resik['Invoice'].nunique()

28882

## Agregat from only one column

In [ ]:
df_resik.loc[df_resik['Description'].str.contains('light', case=False)].groupby('Description')['Quantity'].sum().sort_values(ascending = False)

,Quantity
Description,
white hanging heart t-light holder,100382
hanging heart zinc t-light holder,27356
single heart zinc t-light holder,17920
set/6 pink butterfly t-lights,5948
white cherry lights,4092
set/6 turquoise butterfly t-lights,3836
"feather pen,light pink",3697
pink cherry lights,2974
set of 3 bird light pink feather,2280


In [ ]:
df_resik.loc[df_resik['Description'].str.contains('girl', case=False)].groupby('Description')['Quantity'].sum().sort_values(ascending = False)

,Quantity
Description,
green geisha girl,99
blue geisha girl,97
girls party bag,92
pink baby girls photo album,17
pink baby girl's memory box,2


## Agregat from few Columns

In [ ]:
df_resik.loc[df_resik['Description'].str.contains('light')].groupby('Description', as_index=False) \
.agg(total_quantity=('Quantity', 'sum'), total_amount=('Amount', 'sum')).sort_values('total_amount', ascending=False)

,Description,total_quantity,total_amount
50,white hanging heart t-light holder,100382,276491.71
1,white cherry lights,4092,24469.82
18,hanging heart zinc t-light holder,27356,23051.24
43,single heart zinc t-light holder,17920,17983.15
27,pink cherry lights,2974,17809.28
37,set/6 pink butterfly t-lights,5948,11492.64
20,light pink cherry lights,1891,11183.99
41,set/6 turquoise butterfly t-lights,3836,7676.74
39,set/6 purple butterfly t-lights,2255,5058.15
53,white tall porcelain t-light holder,2110,4639.48
